# inputdata

In [ ]:
pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.neural_network import MLPRegressor
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.model_selection import GridSearchCV, KFold
import statsmodels.api as sm
import statsmodels.api as smapi
import yfinance as yf

In [ ]:
data = yf.download("GBPUSD=X", start='2017-12-03', end='2023-07-10', interval="1d")
data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-12-04,1.349073,1.353858,1.342246,1.349400,1.349400,0
2017-12-05,1.347527,1.348072,1.337614,1.347527,1.347527,0
2017-12-06,1.341562,1.343725,1.336059,1.341526,1.341526,0
2017-12-07,1.339154,1.343111,1.332339,1.339226,1.339226,0
2017-12-08,1.348290,1.351954,1.335827,1.348436,1.348436,0


In [ ]:
data.to_csv('data_crypto.csv')

In [ ]:
# Load data

df_bnb = pd.read_csv("/content/data_crypto.csv", )#index_col=0)
df = pd.read_csv("/content/data_crypto.csv", index_col=0)

df_adj = df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)

In [ ]:
fig = px.line(df_bnb, x="Date", y="Adj Close")
fig.show()

In [ ]:
import yfinance as yf
from datetime import datetime

# Mendapatkan tanggal saat ini
now = datetime.now().strftime("%Y-%m-%d")

# Menarik data kripto dari awal hingga saat ini
realtime_data = yf.download("GBPUSD=X", start='2017-01-01', end=now, interval="1d")
realtime_data.to_csv('realtimebnb.csv')
realtime_bnb = pd.read_csv("/content/realtimebnb.csv", )
fig = px.line(realtime_bnb, x="Date", y="Adj Close")
fig.show()
real_adj = realtime_bnb.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)
real_adj

[*********************100%***********************]  1 of 1 completed


,Date,Adj Close
0,2017-01-02,1.234903
1,2017-01-03,1.228199
2,2017-01-04,1.224560
3,2017-01-05,1.232681
4,2017-01-06,1.242545
...,...,...
1695,2023-07-06,1.270454
1696,2023-07-07,1.274275
1697,2023-07-10,1.283895
1698,2023-07-11,1.286471


In [ ]:
#cek nilai nan
nan_count = data.isna().sum()
# Menampilkan hasil
print(nan_count)

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


# ARIMA

In [ ]:
# Menggunakan Auto ARIMA untuk menentukan parameter terbaik
Parammodel = auto_arima(df_adj, seasonal=False)

# Menampilkan parameter terbaik
print("Best Parameters:", Parammodel.order)#

Best Parameters: (0, 1, 0)


In [ ]:
#SPLITDATA 80:20
data_train, data_test = df_adj[:int(len(df)*0.8)], df_adj[int(len(df)*0.8):]

In [ ]:
pip install statsmodels

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# ARIMA model
arima_model = sm.tsa.ARIMA(df_adj, order=(0,1,0))#4,1,2
# Fit ARIMA model
arima_model_fit = arima_model.fit()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [ ]:
# Prediksi dengan model ARIMA
arima_predictions = arima_model_fit.predict(start=len(data_train), end=len(df_adj)-1)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
arima_mape = mean_absolute_percentage_error(data_test, arima_predictions)
print("ARIMA MAPE:", arima_mape)

ARIMA MAPE: 0.005499676100968691


In [ ]:
# Membuat dataframe untuk plotting
df_plot = pd.DataFrame({'Date': data_train.index, 'Adj Close': data_train['Adj Close']})
df_plott = pd.DataFrame({'Date': data_test.index, 'Adj Close': data_test['Adj Close']})
# Membuat dataframe untuk prediksi
forecast_data = pd.DataFrame({'Date': data_test.index, 'Forecast': arima_predictions[:len(data_test)]})

# Menggabungkan data latihan, data pengujian, dan data prediksi
data = pd.concat([df_plot,df_plott, forecast_data])

# Plot menggunakan Plotly Express
fig = px.line(data, x='Date', y=['Adj Close', 'Forecast'], title='ARIMA Forecast for BNB ')
fig.show()

In [ ]:
print(arima_predictions.shape)

(292,)


In [ ]:
future_predictions_arima = arima_model_fit.forecast(steps=5)  # n adalah jumlah periode yang ingin diprediksi

# Menampilkan prediksi masa depan
print(future_predictions_arima)

1459    1.274275
1460    1.274275
1461    1.274275
1462    1.274275
1463    1.274275
Name: predicted_mean, dtype: float64


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



# TSCV ARIMA

In [ ]:
# Menyiapkan DataFrame untuk menyimpan hasil prediksi dan data asli
results_df = pd.DataFrame({'Index Waktu': df_adj.index, 'Data Asli': df_adj.values.flatten()})

# Inisialisasi array untuk menyimpan hasil prediksi ARIMA
arima_predictions = np.array([])

In [ ]:
n_splits = 5
mape_scores = []
tscv = TimeSeriesSplit(n_splits=n_splits)

# Melakukan TSCV
fold_number = 1
for train_index, test_index in tscv.split(df_adj):
    train_data = df_adj.iloc[train_index]
    test_data = df_adj.iloc[test_index]

    # Fit ARIMA model
    arima_model_fit

    # Melakukan prediksi pada data uji
    arima_fold_predictions = arima_model_fit.predict(start=test_index[0], end=test_index[-1], typ='levels')

    # Menyimpan hasil prediksi pada array
    arima_predictionss = np.concatenate((arima_predictions, arima_fold_predictions))

    # Menghitung skor MAPE
    mape = mean_absolute_percentage_error(test_data, arima_fold_predictions)
    mape_scores.append(mape)
    print("Skor MAPE pada fold ke", fold_number, ":", mape)
    fold_number += 1

Skor MAPE pada fold ke 1 : 0.0035245329662397552
Skor MAPE pada fold ke 2 : 0.004005442310771272
Skor MAPE pada fold ke 3 : 0.002802865266096221
Skor MAPE pada fold ke 4 : 0.002978719181340041
Skor MAPE pada fold ke 5 : 0.0032953094310047637


In [ ]:

# Menampilkan skor MAPE rata-rata
average_mape = np.nanmean(mape_scores)
print("Average MAPE:", average_mape)

# Menyimpan skor MAPE pada DataFrame
results_df['Prediksi ARIMA'] = np.nan
results_df.iloc[-len(arima_predictionss):, results_df.columns.get_loc('Prediksi ARIMA')] = arima_predictionss

Average MAPE: 0.0033213738310904108


In [ ]:
# Plot hasil prediksi dan data asli
fig = px.line(results_df, x='Index Waktu', y=['Data Asli', 'Prediksi ARIMA'], title='Prediksi ARIMA')
fig.show()

#ANN

In [ ]:
data = pd.read_csv("/content/data_crypto.csv", index_col=0)
df_adj = data.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)

In [ ]:
#memastikan demensi data sama dengan reshape
dataset_awal = df_adj['Adj Close'].values
dataset_reshape = dataset_awal.reshape(-1, 1)

# Normalisasi data menggunakan MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset_reshape)

# Membagi data menjadi data latih dan data uji
train_data = dataset[:int(0.8 * len(dataset))]
test_data = dataset[int(0.8 * len(dataset)):]



In [ ]:
# Membuat dataset latih dengan jendela waktu
def create_dataset(dataset, window_size):
    X, Y = [], []
    for i in range(len(dataset)-window_size-1):
        X.append(dataset[i:(i+window_size), 0])
        Y.append(dataset[i + window_size, 0])
    return np.array(X), np.array(Y)

window_size = 1  # Ukuran jendela waktu (window size)
X_train, y_train = create_dataset(train_data, window_size)
X_test, y_test = create_dataset(test_data, window_size)

In [ ]:
df_adj['Adj Close']

Date
2017-12-04    1.349400
2017-12-05    1.347527
2017-12-06    1.341526
2017-12-07    1.339226
2017-12-08    1.348436
                ...   
2023-07-03    1.269325
2023-07-04    1.269293
2023-07-05    1.271763
2023-07-06    1.270454
2023-07-07    1.274275
Name: Adj Close, Length: 1459, dtype: float64

In [ ]:
print("Data awal:")
print(dataset_awal[:5])


Data awal:
[1.34940028 1.34752727 1.34152567 1.33922589 1.34843588]


In [ ]:
print("Data setelah reshape:")
print(dataset_reshape[:5])

Data setelah reshape:
[[1.34940028]
 [1.34752727]
 [1.34152567]
 [1.33922589]
 [1.34843588]]


In [ ]:
print("Data setelah minmaxscler:")
print(dataset[:5])


Data setelah minmaxscler:
[[0.76537527]
 [0.76019335]
 [0.74358921]
 [0.73722657]
 [0.76270714]]


In [ ]:
print("Dimensi sebelum reshape:", dataset_awal.shape)
print("Dimensi sesudah reshape:", dataset_reshape.shape)

Dimensi sebelum reshape: (1459,)
Dimensi sesudah reshape: (1459, 1)


In [ ]:
# Fungsi untuk membuat model ANN dengan hyperparameter yang akan dioptimasi
def create_model(units=10, learning_rateANN=0.01, dropout_rateANN=0.0):
    model = Sequential()
    model.add(Dense(units=units, input_dim=window_size, activation='relu'))
    model.add(Dropout(dropout_rateANN))
    model.add(Dense(units=1, activation='linear'))
    optimizer = Adam(learning_rate=learning_rateANN)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

In [ ]:
# Membungkus model ke dalam KerasRegressor
estimator = KerasRegressor(build_fn=create_model, verbose=0)

<ipython-input-27-db55ed7821ac>:2: DeprecationWarning:

KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.



In [ ]:
# Definisikan hyperparameter yang akan dioptimasi
param_grid = {
    'units': [10,30,50],#, 30,50
    'epochs': [100, 150,250],#, 150,250
    'batch_size': [ 16, 32,64],#16, 32,
}

# Buat objek TimeSeriesSplit untuk k-fold cross-validation
tscv = TimeSeriesSplit(n_splits=5)
#Best Parameters: {'batch_size': 64, 'epochs': 100, 'units': 10}
#Best MSE Score: 0.0021351799721713873

In [ ]:
# Buat objek GridSearchCV
grid_search = GridSearchCV(estimator=estimator, param_grid=param_grid, scoring='neg_mean_squared_error', cv=tscv)
# Melakukan grid search pada data latih
grid_search.fit(X_train, y_train)


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7ad09cd294e0>,
             param_grid={'batch_size': [16, 32, 64], 'epochs': [100, 150, 250],
                         'units': [10, 30, 50]},
             scoring='neg_mean_squared_error')

In [ ]:
# Menampilkan hasil grid search
print('Best Parameters:', grid_search.best_params_)
print('Best MSE Score:', -grid_search.best_score_)

Best Parameters: {'batch_size': 16, 'epochs': 100, 'units': 30}
Best MSE Score: 0.0003950000953987485


In [ ]:
# Mendapatkan model terbaik dari grid search
best_model = grid_search.best_estimator_.model

In [ ]:
# Early stopping
es = EarlyStopping(monitor='val_loss', mode='min', patience=5, verbose=0)

In [ ]:
# Melatih model dengan model terbaik dan early stopping
best_model.fit(X_train, y_train, validation_split=0.3, callbacks=[es], verbose=0)

# Memprediksi data latih dan data uji dengan model terbaik
train_predictions = best_model.predict(X_train)
test_predictions = best_model.predict(X_test)

10/10 [==============================] - 0s 2ms/step


In [ ]:
print(test_predictions.shape)

(290, 1)


In [ ]:
# Menghitung MAPE untuk data latih dan data uji
def calculate_mape(actual, predicted):
    return np.mean(np.abs((actual - predicted) / actual)) * 100

train_mape = calculate_mape(y_train, train_predictions.flatten())
test_mape = calculate_mape(y_test, test_predictions.flatten())

print('MAPE - Data Latih:', train_mape)
print('MAPE - Data Uji:', test_mape)

MAPE - Data Latih: 2.3897779385368803
MAPE - Data Uji: inf


<ipython-input-35-800d547bd8d5>:3: RuntimeWarning:

divide by zero encountered in true_divide



In [ ]:


# Mengubah data yang sudah dinormalisasi ke dalam skala semula
train_predictions = scaler.inverse_transform(train_predictions.reshape(-1, 1))
y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
ann_predictions = scaler.inverse_transform(test_predictions.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Menyesuaikan dimensi array
train_predictions = train_predictions.flatten()
y_train = y_train.flatten()
ann_predictions = ann_predictions.flatten()
y_test = y_test.flatten()



In [ ]:
# Membuat dataframe untuk data aktual dan prediksi
df_train = pd.DataFrame({'x': np.arange(len(y_train)), 'Actual Train Data': y_train, 'Train Predictions': train_predictions})
df_test = pd.DataFrame({'x': np.arange(len(y_train), len(y_train) + len(y_test)), 'Actual Test Data': y_test, 'Test Predictions': ann_predictions})

# Menggabungkan data train dan test dalam satu dataframe
df = pd.concat([df_train, df_test], axis=0)

# Menampilkan grafik hasil prediksi menggunakan plotly.express
fig = px.line(df, x='x', y=['Actual Train Data', 'Train Predictions', 'Actual Test Data', 'Test Predictions'],
              labels={'value': 'Price'})
fig.update_layout(title='MODEL ANN')
fig.show()

# HYBRID ARIMA-ANN (META MODEL)

In [ ]:
print("panjang awal ARIMA predik: ",arima_predictions.shape)
print("============")
print("panjang awal ANN PREDIK: ",ann_predictions.shape)

panjang awal ARIMA predik:  (292,)
panjang awal ANN PREDIK:  (290,)


In [ ]:
arima_predictions_trimmed = arima_predictions[-len(ann_predictions):]  # Trim prediksi ARIMA menjadi ukuran yang sama dengan prediksi ANN
hybrid_predictions = arima_predictions_trimmed + ann_predictions[:len(arima_predictions_trimmed)]

In [ ]:
alpha = 0.7 # Bobot untuk prediksi ARIMA
trimmed_ann_predictions_bobot = ann_predictions[:len(arima_predictions_trimmed)]
hybrid_predictions = alpha * arima_predictions_trimmed + (1 - alpha) * trimmed_ann_predictions_bobot

In [ ]:
print("panjang BARU ARIMA predik: ",arima_predictions_trimmed.shape)
print("============")
print("panjang BARU ANN predik: ",trimmed_ann_predictions_bobot.shape)

panjang BARU ARIMA predik:  (290,)
panjang BARU ANN predik:  (290,)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Menghitung RMSE
rmse_before = mean_squared_error(arima_predictions_trimmed, trimmed_ann_predictions_bobot, squared=False)
rmse_after = mean_squared_error(hybrid_predictions, trimmed_ann_predictions_bobot, squared=False)

# Menghitung MAE
mae_before = mean_absolute_error(arima_predictions_trimmed, trimmed_ann_predictions_bobot)
mae_after = mean_absolute_error(hybrid_predictions, trimmed_ann_predictions_bobot)

# Menampilkan hasil
print("RMSE sebelum diberikan bobot:", rmse_before)
print("RMSE setelah diberikan bobot:", rmse_after)
print("MAE sebelum diberikan bobot:", mae_before)
print("MAE setelah diberikan bobot:", mae_after)


RMSE sebelum diberikan bobot: 0.011846034372087186
RMSE setelah diberikan bobot: 0.008292220404662567
MAE sebelum diberikan bobot: 0.008065200263056259
MAE setelah diberikan bobot: 0.005645637923273553


In [ ]:
# Fungsi untuk membuat model meta
def create_meta_model2(learning_rate2=0.01, dropout_rate2=0.001,batch_size2='batch_size'):
    meta_model2 = Sequential()
    meta_model2.add(Dense(10, input_dim=2, activation='relu'))  # Input layer dengan 2 fitur: prediksi ARIMA dan prediksi ANN
    meta_model2.add(Dropout(dropout_rate2))
    meta_model2.add(Dense(1))
    optimizer2 = Adam(learning_rate=learning_rate2)
    meta_model2.compile(loss='mean_squared_error', optimizer=optimizer2)
    return meta_model2

In [ ]:
if len(ann_predictions) > len(arima_predictions_trimmed):
    trimmed_ann_predictions_bobot = ann_predictions[:len(arima_predictions_trimmed)]

X_train_meta2 = np.column_stack((arima_predictions_trimmed, trimmed_ann_predictions_bobot))
y_train_meta2 = trimmed_ann_predictions_bobot


In [ ]:
print(X_train_meta2.shape)
print(X_train_meta2)

(290, 2)
[[1.26173413 1.25745714]
 [1.2628336  1.2605468 ]
 [1.26470208 1.26163423]
 [1.26089096 1.26348233]
 [1.24831474 1.25971282]
 [1.25792491 1.24727368]
 [1.24923015 1.25677907]
 [1.25234818 1.24817908]
 [1.25873244 1.25126314]
 [1.25402856 1.25757778]
 [1.24929726 1.25292516]
 [1.2274909  1.2482456 ]
 [1.21447659 1.22667694]
 [1.2012589  1.2138046 ]
 [1.21743369 1.20322311]
 [1.23548305 1.2167294 ]
 [1.22282279 1.23458195]
 [1.2251749  1.22205973]
 [1.22666276 1.22438622]
 [1.2250849  1.22585785]
 [1.22607625 1.22429717]
 [1.22824466 1.22527766]
 [1.22673798 1.22742248]
 [1.21897697 1.22593224]
 [1.21317995 1.21825588]
 [1.21608639 1.21275127]
 [1.21027279 1.21539676]
 [1.21144569 1.21042764]
 [1.19491446 1.2113651 ]
 [1.19232142 1.1981523 ]
 [1.20280492 1.19607985]
 [1.20166302 1.20445883]
 [1.19009364 1.20354617]
 [1.18777537 1.19429922]
 [1.18638039 1.19244635]
 [1.18396437 1.19133139]
 [1.18883443 1.18940032]
 [1.19482875 1.19329274]
 [1.20035052 1.19808388]
 [1.19717467 1.2

In [ ]:
"""print(y_train_meta2.shape)
print(y_train_meta2)"""

'print(y_train_meta2.shape)\nprint(y_train_meta2)'

In [ ]:
# Membuat objek GridSearchCV untuk model meta
meta_estimator2 = MLPRegressor()


meta_param_grid2 = {
    'hidden_layer_sizes': [(10,)],#(10,),(20,),(30,),,(60,)
    'learning_rate': ['constant', 'adaptive'],
    'alpha': [0.7],#0.6,,0.8,0.9
    'random_state': [60],#,40,50,60
    'batch_size': [16]#,32,64
}
"""
Best Parameters (Meta Model): {'alpha': 0.7, 'batch_size': 16, 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'random_state': 20}
Best MSE Score (Meta Model): 0.3153940456820578"""

"\nBest Parameters (Meta Model): {'alpha': 0.7, 'batch_size': 16, 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'random_state': 20}\nBest MSE Score (Meta Model): 0.3153940456820578"

In [ ]:
meta_grid_search2 = GridSearchCV(estimator=meta_estimator2, param_grid=meta_param_grid2, scoring='neg_mean_squared_error', cv=5)

In [ ]:
# Melakukan grid search untuk model meta
meta_grid_search2.fit(X_train_meta2, y_train_meta2)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning:

Stochastic Optimizer: Maximum

GridSearchCV(cv=5, estimator=MLPRegressor(),
             param_grid={'alpha': [0.7], 'batch_size': [16],
                         'hidden_layer_sizes': [(10,)],
                         'learning_rate': ['constant', 'adaptive'],
                         'random_state': [60]},
             scoring='neg_mean_squared_error')

In [ ]:
print('Best Parameters (Meta Model):', meta_grid_search2.best_params_)
print('Best MSE Score (Meta Model):', -meta_grid_search2.best_score_)

Best Parameters (Meta Model): {'alpha': 0.7, 'batch_size': 16, 'hidden_layer_sizes': (10,), 'learning_rate': 'constant', 'random_state': 60}
Best MSE Score (Meta Model): 0.0008849003269614585


In [ ]:
# Menggunakan model meta terbaik untuk memprediksi
best_meta_model2= meta_grid_search2.best_estimator_
hybrid_predictions = hybrid_predictions[:len(arima_predictions_trimmed)]
X_test_meta2 = np.column_stack((hybrid_predictions, trimmed_ann_predictions_bobot))
hybrid_predictions_final = best_meta_model2.predict(X_test_meta2)
print(hybrid_predictions_final)


[1.20707651 1.20750798 1.2076973  1.2078424  1.20703194 1.20564971
 1.20667178 1.205627   1.20619136 1.20689745 1.20616995 1.20500723
 1.20186019 1.1998442  1.19887065 1.20109162 1.20310401 1.20152742
 1.201869   1.20202138 1.20184254 1.20202547 1.2022676  1.20187667
 1.20072717 1.20008146 1.20028012 1.19966054 1.19936507 1.19757322
 1.1975675  1.19863337 1.19822162 1.19695487 1.19667752 1.19647076
 1.19634159 1.1970017  1.19776728 1.19826359 1.19799368 1.19822537
 1.1983017  1.19896772 1.19909588 1.20048759 1.20060618 1.2007471
 1.20158746 1.20014127 1.20014312 1.20014032 1.19913428 1.19937098
 1.1996121  1.20119003 1.20080011 1.19977868 1.19914094 1.19943791
 1.19859982 1.19713623 1.19580853 1.19543736 1.19602571 1.19567888
 1.19568832 1.19440704 1.19454045 1.19373043 1.19296538 1.19225988
 1.19171575 1.1925368  1.19198683 1.19207445 1.19234043 1.19353954
 1.19381008 1.19199091 1.19216262 1.19114956 1.19097129 1.19091811
 1.18994887 1.1886971  1.18758805 1.18531627 1.18510467 1.18530

In [ ]:
import plotly.express as px

# Menampilkan prediksi data train
train_indices = range(len(train_predictions) + len(trimmed_ann_predictions_bobot))
test_indices = range(len(train_predictions), len(train_predictions) + len(trimmed_ann_predictions_bobot))


# Mengubah range menjadi list
train_indices = list(range(len(train_predictions)+ len(trimmed_ann_predictions_bobot)))
test_indices = list(range(len(train_predictions), len(train_predictions) + len(trimmed_ann_predictions_bobot)))

fig_train = px.line()
fig_train.add_scatter(x=train_indices, y=train_predictions, name='Train')
fig_train.add_scatter(x=test_indices, y=trimmed_ann_predictions_bobot, name='Actual')
fig_train.add_scatter(x=test_indices, y=hybrid_predictions_final, name='Hybrid')
fig_train.update_layout(xaxis_title='Time', yaxis_title='Value', title='Hybrid Model - Train Predictions')
fig_train.show()

fig_test = px.line()
fig_test.add_scatter(x=test_indices, y=trimmed_ann_predictions_bobot, name='Actual')
fig_test.add_scatter(x=test_indices, y=hybrid_predictions_final, name='Hybrid')
fig_test.update_layout(xaxis_title='Time', yaxis_title='Value', title='Hybrid Model - Test Predictions')
fig_test.show()


In [ ]:
import numpy as np
def calculate_mape(actual, predicted):
    return np.mean(np.abs((actual - predicted) / actual)) * 100

# Menghitung MAPE untuk data uji
"""test_mape = calculate_mape(y_test[0], hybrid_predictionss.flatten())"""
test_mape2 = calculate_mape(np.transpose(trimmed_ann_predictions_bobot[:len(hybrid_predictions_final)]), hybrid_predictions_final)


print('MAPE - Data Uji META MODEL 2:', test_mape2)

MAPE - Data Uji META MODEL 2: 2.2160871418537456


# ESTIMASI 1


In [ ]:
# Menggunakan model meta terbaik untuk memprediksi
best_meta_model2 = meta_grid_search2.best_estimator_
hybrid_predictions = hybrid_predictions[:len(arima_predictions_trimmed)]
X_test_meta2 = np.column_stack((hybrid_predictions, ann_predictions))
hybrid_predictions_finals = best_meta_model2.predict(X_test_meta2)
#print(hybrid_predictions_final)

In [ ]:
forecast_dates = pd.date_range(start='2023-05-02', periods=len(hybrid_predictions_finals))

forecast_dates = forecast_dates[:200]
hybrid_predictions_finals = hybrid_predictions_finals[:200]


forecast_df = pd.DataFrame({
    'Date': forecast_dates,
    'Forecast': hybrid_predictions_finals
})
forecast_df.head(5)

,Date,Forecast
0,2023-05-02,1.207077
1,2023-05-03,1.207508
2,2023-05-04,1.207697
3,2023-05-05,1.207842
4,2023-05-06,1.207032


In [ ]:
import plotly.express as px

fig = px.line(forecast_df, x='Date', y='Forecast', title='Forecast')
fig.show()
#============================

# Membuat DataFrame yang difilter berdasarkan rentang tanggal tertentu
start_date = '2023-05-02'
end_date = '2023-11-10'
filtered_data = realtime_bnb[(realtime_bnb['Date'] >= start_date) & (realtime_bnb['Date'] <= end_date)]
fig = px.line(filtered_data, x="Date", y="Adj Close", title='realtime Data filter')
fig.show()
#============================
fig = px.line(realtime_bnb, x="Date", y="Adj Close",title='realtime Data no cut')
fig.show()

In [ ]:
forecast_df.head(5)


,Date,Forecast
0,2023-05-02,1.207077
1,2023-05-03,1.207508
2,2023-05-04,1.207697
3,2023-05-05,1.207842
4,2023-05-06,1.207032


In [ ]:
data_test.head(5)

,Adj Close
Date,
2022-05-26,1.258610
2022-05-27,1.261734
2022-05-30,1.262834
2022-05-31,1.264702
2022-06-01,1.260891
